In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import BertModel
model =BertModel.from_pretrained("cl-tohoku/bert-base-japanese-v2")

/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/text/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initia

In [2]:
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-v2')

In [3]:
df_train = pd.read_csv('/home/yamanishi/project/trip_recommend/model/recommendation/data/df_review_train.csv')

In [4]:
import torch
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
        self.review = df['review'].values
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        review = self.review[index]
        review = self.tokenizer.encode_plus(review, max_length=512, pad_to_max_length=True)
        return torch.tensor(review['input_ids']), torch.tensor(review['attention_mask'])
        

In [32]:
df_exp=pd.read_csv('/home/yamanishi/project/trip_recommend/data/jalan/spot/experience_light.csv')

In [5]:
dataset = ReviewDataset(df_train)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=False)

In [26]:
dataset[1199387]

(tensor([    2,   860,  5333,   862, 20091, 11175,   828,  4901,  8237,   862,
         13034, 11152,   828, 11380,   893, 11170,   881,   860,  5333,   932,
         16809, 11143,   862, 11187, 13276,   881,   829,  1154,   890, 12725,
           888,   916,  3946,   897, 15150,   862, 13592, 24892, 17742,   881,
         12461,   829, 11168, 12419, 11722,   892,    28,    28,    28,    15,
             3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [6]:
from tqdm import tqdm
device = 'cuda:9'
model.to(device)
output_all = []
with torch.no_grad():
    for data in tqdm(dataloader):
        input_ids, mask = data[0], data[1]
        output=model(input_ids.to(device), mask.to(device), return_dict=False)
        output = torch.mean(output[0], dim=1)
        output_all.append(output)

  0%|          | 0/11994 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/text/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|████

In [21]:
len(output_all)

11994

In [20]:
output_all_cat = torch.cat(output_all, dim=0).to('cpu')
print(output_all_cat.size())
torch.save(output_all_cat,'/home/yamanishi/project/trip_recommend/model/recommendation/data/bert_output.pt')

torch.Size([1199400, 768])


In [30]:
df_user = pd.read_csv('/home/yamanishi/project/trip_recommend/model/recommendation/data/user_attributes.csv')
len(df_user)

27094

In [35]:
user_sca = [0]*1199388
spot_sca = [0]*1199388
url_id = {url:id for id,url in enumerate(df_user['url'])}
spot_id = {spot_name:id for id,spot_name in enumerate(df_exp['spot_name'])}
for i,(url, spot) in enumerate(zip(df_train['url'], df_train['spot'])):
    user_sca[i] = url_id[url]
    spot_sca[i] = spot_id[spot]


In [41]:
user_out = torch.zeros(27904, 768)
spot_out = torch.zeros(42852, 768)
user_rep = torch.scatter(output_all_cat[:1199388],dim=1, index=torch.tensor(user_sca), out=user_out)
print(user_rep.shape) 

TypeError: scatter() received an invalid combination of arguments - got (Tensor, out=Tensor, index=Tensor, dim=int), but expected one of:
 * (Tensor input, int dim, Tensor index, Tensor src, *, Tensor out)
 * (Tensor input, int dim, Tensor index, Tensor src, *, str reduce, Tensor out)
 * (Tensor input, name dim, Tensor index, Tensor src)
      didn't match because some of the keywords were incorrect: out
 * (Tensor input, int dim, Tensor index, Number value, *, Tensor out)
 * (Tensor input, int dim, Tensor index, Number value, *, str reduce, Tensor out)
 * (Tensor input, name dim, Tensor index, Number value)
      didn't match because some of the keywords were incorrect: out


In [18]:
output_all[-1] = o

In [ ]:
df_user = pd.read_csv('')

In [ ]:
user_id = {user:id for user,id in enumerate()}

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity 
cos_sim = cosine_similarity(output.cpu().detach().numpy())
cos_sim

array([[0.9999995 , 0.91862714, 0.91664577, ..., 0.8930747 , 0.8908178 ,
        0.89602804],
       [0.91862714, 0.99999946, 0.92893904, ..., 0.88491714, 0.90921235,
        0.92315704],
       [0.91664577, 0.92893904, 0.9999998 , ..., 0.8775908 , 0.908112  ,
        0.9197801 ],
       ...,
       [0.8930747 , 0.88491714, 0.8775908 , ..., 1.0000001 , 0.8964453 ,
        0.8940555 ],
       [0.8908178 , 0.90921235, 0.908112  , ..., 0.8964453 , 0.99999946,
        0.91953045],
       [0.89602804, 0.92315704, 0.9197801 , ..., 0.8940555 , 0.91953045,
        0.9999999 ]], dtype=float32)